# <font color='blue'> ColorBayes: </font>
ColorBayes is an algorithm based on Bayesian inference that corrects local color distortions generated by artifitial light (LED) in high-throughput plant phenotyping image. The algorithm estimates the local illuminants using the Bayes' rule, the maximum a posteriori, the observed image data, and prior illuminant information. <br/>


**Assumption:** 

- An observed image is divided into areas that correspond to individual pot areas.
- The objects of a pot area are segmented, such as leaves and soil.
- A pixel class is assigned individually to a segmented object; for instance, the plant pixel class contains the leaves, stem, and other plant organs. A class is a collection of $n$ pixels  $Z={z_i }$, where $z_i$ is a random variable representing the pixel value at location $i=0,1,2,…,n$.
- The reflectance of a pixel class is a collection of reflectances $R={r_i }$, where r_i is a random variable representing the reflectance at the location $i=0,1,2,…,n$. Reflectances are independent to each other such as $p(r_i,r_j )=p(r_i )p(r_j )$. Based on the independence assumption, we have $p(R)= \sum_{i=1}^{n} p(r_i )$ 
- The illumination $(l)$ has a uniform probability distribution over a pixel class $p(L)=u$ where $u$ is constant. 
- The illumination and the reflectance are statistically independent of each other $p(L,R)=p(L)p(R)$.
- A pixel value z_i is a function of the reflectance r_i, illuminant l and additive Gaussian noise w_i. This noise has a mean equal to zero and a standard deviation σ. <br/>
$z_i=lr_i+w_i$  (Eq. 1).

### Likelihood: 
The likelihood of the pixel class is given the illuminant and reflectance and follows a normal distribution. <br/>
$$p(Z│L,R)= \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi \sigma^2}}  exp⁡\biggl(-\frac{(z_i-lr_i)^2}{2\sigma^2}\biggr)$$  


### Priors: Reflectance & Illuminant: 
We created an **image dataset** to get the reflectance and illuminant prior distributions. It has images of green fabric pieces on pots and Macbeth colorChecker charts. They were illuminated using D65 standard illuminant.. <br/>

$$P(R)= \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi \tau^2}}  exp⁡\biggl(-\frac{(r_i-\mu)^2}{2\tau^2}\biggr)$$ <br/>
As the illumation is uniform over a pixel class the probability distribution is given by:
$$p(L)=u$$ <br/>

### Posterior
It is possible to analytically calculate the posterior distribution using the Bayes' rule as the prior is a conjugate prior for the likelihood function. The posterior distribution is given by:
$$P(L|Z)=\prod_{i=1}^{n} \int \frac{1}{\sqrt{2\pi \sigma^2}}  exp⁡\biggl(-\frac{(z_i-lr_i)^2}{2\sigma^2}\biggr) \frac{1}{\sqrt{2\pi \tau^2}}  exp⁡\biggl(-\frac{(r_i-\mu)^2}{2\tau^2}\biggr) d r_i $$ <br/>


### Maximum a posteriori 
We estimate the illumination value when the posterior distribution reaches the highest value.

$$  \hat{l}_{MAP} =\underset{l}{\operatorname{argmax}}  P(L|Z) = \frac{\sum_{i=1}^{n}z_i} {n \mu} $$





### 3.1 Import &rarr; Python modules
The Bayesian color correction program needs multiple Python modules to run properly

In [1]:
import os
import sys
import cv2
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pylab as plt
from scipy.stats import shapiro
from scipy.stats import normaltest
from IPython.display import display, HTML
from statsmodels.graphics.gofplots import qqplot

### 3.2 Import &rarr; User modules
We created our own modules to organize the code and follow the object oriented programming philosophy

In [2]:
import save
import imalgo
import plotting
import cropping
import leafcount
import colorstacs
import segmentation
import import_ipynb
import imgCorrection

PermissionError: [WinError 5] Access is denied: 'C:/Users/17600112'

### 3.3 Variable declaration 
Define the Experiment & camera ID details

In [ ]:
cardLoc =  ['left', 'right']
expID = 'cloth'
cam = 'cam03'
winsiz = 300
win = int(winsiz/2)

Define the folder paths where is located the green fabric dataset

In [ ]:
dirParent   = "C:/Users/17600112/OneDrive - LA TROBE UNIVERSITY/PhD - Datasets/colour_constancy/"
root        = os.path.join(dirParent, expID)
path_label  = os.path.join(root, expID + '_label.csv')
path_coor   = os.path.join(root, expID + '_coord_' + cam + '.csv')
path_mtx    = os.path.join(root, 'camParam' , cam + '_MTX' + '.csv')
path_dis    = os.path.join(root, 'camParam' , cam + '_DIST' + '.csv')
path_pers   = os.path.join(root, 'camParam' , cam + '_PERS' + '.csv')
path_CSV    = os.path.join(root, expID + '_csvFiles')
path_trash  = os.path.join(dirParent, 'trash')
path_trashCorr  = os.path.join(path_trash, 'correct')
path_mask       = os.path.join(root, expID + '_mask')
path_checker    = os.path.join(root, expID + '_checker')
path_img = os.path.join(root, expID + '_' + 'images')
path_imagesL = os.path.join(path_img, cam + '_' + cardLoc[0])
path_imagesR = os.path.join(path_img, cam + '_' + cardLoc[1])

if not os.path.exists(path_CSV): os.makedirs(path_CSV) 
if not os.path.exists(path_trash): os.makedirs(path_trash) 
if not os.path.exists(path_mask): os.makedirs(path_mask) 
if not os.path.exists(path_trashCorr): os.makedirs(path_trashCorr) 
if not os.path.exists(path_checker): os.makedirs(path_checker) 

### 3.4 Read files & set Variables

Parameters for the image prespective and lens

In [ ]:
mtx = pd.read_csv(path_mtx, sep=',',header=None).values
dist = pd.read_csv(path_dis, sep=',',header=None).values
pers = pd.read_csv(path_pers, sep=',',header=None).values
coor = pd.read_csv(path_coor, sep=',')

Get the list of images which were taken using the two DSLR cameras (left & right).

In [ ]:
imagesL = os.listdir(path_imagesL) 
imagesR = os.listdir(path_imagesR) 
df_main = pd.read_csv(path_coor)
df_main['y_dist'] = np.sqrt((df_main.loc[:,'y_centre'] - (df_main.loc[:,'y_centre'].max() - df_main.loc[:,'y_centre'].min())/2)**2)
df_main['x_dist'] = np.sqrt((df_main.loc[:,'x_centre'] - (df_main.loc[:,'x_centre'].max() - df_main.loc[:,'x_centre'].min())/2)**2)

### 3.5  Illuminant distribution on the image
Load a CSV file that contains the filename, ID, position of the Machbeth colorChecker chart on the image

In [ ]:
checkerlist = pd.DataFrame(os.listdir(path_checker), columns=['filename'])
checkerlist[['imgName', 'position', 'name']]= checkerlist.loc[:, 'filename'].str.split("_", expand = True) 
checkerlist.loc[:, 'name'] = checkerlist.loc[:, 'name'].apply(lambda x: x[:-4])

###  Function &rarr; illuminant_colorChart

1. Read a file that contains the location, light intensity and name of each Macbeth card on the image
2. Load 180 Macbeth cardChecker images 
3. Get **samples** of each 24 painted patches of each Macbeth cardChecker images
4. Estimate the **Illuminants** using the nonlinear regression (non-linear least square)
5. Estimate the **Alpha** (correction parameter) using the nonlinear regression
6. Calculate the **Mean Square Error** between the ideal values of the Macbeth colorChecker card and the response on the image **before & after** the color correction

In [ ]:
checkers = imgCorrection.illuminant_colorChart(checkerlist, df_main)
display(HTML(checkers[:3].to_html()))
# imgCorrection.plot_illum(checkers)

### Function: RMSE (The Best illumination)
0. Load the Checker matrix
1. Load the MacBeth card images
2. Get pixels **samples** from each patch of the Macbeth cardChecker images (mode)
3. Correct the image by multiplying the **sample by Alpha** 
4. Calculate the Mean Square Error between the ideal values and corrected image
5. Sort and store the MSE results of each color card correcte by Alpha

In [ ]:
checkersRMSE = imgCorrection.RMSE(checkers)
# display(HTML(checkersRMSE.to_html()))

### Reference image of the Macbeth colorChecker chart
Load an image of the Machbeth card that the lowest error reference image of the green fabric dataset that it was illuminated under the illuminant C (approx.)

In [ ]:
imgFile = '01.jpg'
imPathL = os.path.join(path_imagesL, imgFile)    
imPathR = os.path.join(path_imagesR, imgFile)    
imSrcblGR_L = cv2.imread(imPathL,cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
imSrcblGR_R = cv2.imread(imPathR,cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
imGeoBGR_L = imgCorrection.geometryCorr(imSrcblGR_L, mtx, dist, pers)
imGeoBGR_R = imgCorrection.geometryCorr(imSrcblGR_R, mtx, dist, pers)
checkImg =checkersRMSE.loc[checkersRMSE['imgName'] == '01'].copy().reset_index(drop=True)

Create and define variables (empty)

In [ ]:
bluleafR, grnleafR, redleafR = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
bluleafL, grnleafL, redleafL = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
hueleafR, satleafR, valleafR = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
hueleafL, satleafL, valleafL = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
bluleafAuxR, grnleafAuxR, redleafAuxR = [], [], []
bluleafAuxL, grnleafAuxL, redleafAuxL = [], [], []
hueleafAuxR, satleafAuxR, valleafAuxR = [], [], []
hueleafAuxL, satleafAuxL, valleafAuxL = [], [], []

Obtain the colors from the Macbeth chart

In [ ]:
for cntCheck in range(len(checkImg)):
# for cntCheck in range(1):
    imCorBGR_L, imCorBGR_R = [], []
    alphCheck, posnCheck = [], []
    alphCheck = checkImg.loc[cntCheck, ['bluAlpha', 'grnAlpha', 'redAlpha']].values
    posnCheck = checkImg.loc[cntCheck, 'position']
    imNamCheck = checkImg.loc[cntCheck, 'imgName']
    
    imCorBGR_L = imgCorrection.correct(imGeoBGR_L, alphCheck)
    imCorBGR_R = imgCorrection.correct(imGeoBGR_R, alphCheck)